In [ ]:
import boto3
import json
import json
import os

In [ ]:
# Configurações iniciais
lambda_client = boto3.client('lambda')
apigateway_client = boto3.client('apigateway')

function_name = 'funcaoproxy-AndreySilva'
role_arn = 'arn:aws:iam:********************************************' 
region = 'us-east-1'

# Código da função Lambda (versão desenvolvimento)
lambda_code = """


def lambda_handler(event, context):
    try:
        stage = event['requestContext']['stage']
    except KeyError:
        stage = 'desconhecido'

    response_body = {
        'message': f'Olá do ambiente Desenvolvimento {stage}!',
        'functionVersion': context.function_version,
        'functionAlias': (
            context.invoked_function_arn.split(':')[-1]
            if ':' in context.invoked_function_arn else '$LATEST'
        )
    }

    return {
        'statusCode': 200,
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Origin': '*'
        },
        'body': json.dumps(response_body, ensure_ascii=False)
    }
"""

# 1. Criar função Lambda
response = lambda_client.create_function(
    FunctionName=function_name,
    Runtime='python3.9',
    Role=role_arn,
    Handler='lambda_function.lambda_handler',
    Code={'ZipFile': bytes(lambda_code, 'utf-8')},
    Architectures=['x86_64'],
    Publish=False
)
print("Função Lambda criada.")

# 2. Publicar versão 1 (Desenvolvimento)
version_dev = lambda_client.publish_version(FunctionName=function_name)['Version']
print(f"Versão de desenvolvimento publicada: {version_dev}")

# 3. Criar alias 'dev'
lambda_client.create_alias(
    FunctionName=function_name,
    Name='dev',
    FunctionVersion=version_dev,
    Description='Alias para ambiente de desenvolvimento'
)
print("Alias 'dev' criado.")

# 4. Atualizar código para produção
lambda_code_prod = lambda_code.replace('Desenvolvimento', 'Producao')
lambda_client.update_function_code(
    FunctionName=function_name,
    ZipFile=bytes(lambda_code_prod, 'utf-8')
)
print("Código atualizado para produção.")

# 5. Publicar versão 2 (Produção)
version_prod = lambda_client.publish_version(FunctionName=function_name)['Version']
print(f"Versão de produção publicada: {version_prod}")

# 6. Criar alias 'prod'
lambda_client.create_alias(
    FunctionName=function_name,
    Name='prod',
    FunctionVersion=version_prod,
    Description='Alias para ambiente de produção'
)
print("Alias 'prod' criado.")

# 7. Criar API Gateway REST
api_name = 'minha-api-proxy-lab-andrey'
api = apigateway_client.create_rest_api(
    name=api_name,
    description='API Gateway (proxy) para lab de Lambda com Aliases',
    endpointConfiguration={'types': ['REGIONAL']}
)
api_id = api['id']
print(f"API criada: {api_id}")

# 8. Obter ID do recurso raiz
resources = apigateway_client.get_resources(restApiId=api_id)
root_id = next(r['id'] for r in resources['items'] if r['path'] == '/')

# 9. Criar recurso /hello
hello_resource = apigateway_client.create_resource(
    restApiId=api_id,
    parentId=root_id,
    pathPart='hello'
)
hello_id = hello_resource['id']
print("Recurso /hello criado.")

# 10. Criar método GET com integração proxy para 'dev'
lambda_arn_dev = f'arn:aws:lambda:{region}:SEU_ID_DE_CONTA:function:{function_name}:dev'
apigateway_client.put_method(
    restApiId=api_id,
    resourceId=hello_id,
    httpMethod='GET',
    authorizationType='NONE'
)
apigateway_client.put_integration(
    restApiId=api_id,
    resourceId=hello_id,
    httpMethod='GET',
    type='AWS_PROXY',
    integrationHttpMethod='POST',
    uri=f'arn:aws:apigateway:{region}:lambda:path/2015-03-31/functions/{lambda_arn_dev}/invocations'
)
print("Método GET integrado com alias 'dev'.")

# 11. Implantar estágio Desenvolvimento
apigateway_client.create_deployment(
    restApiId=api_id,
    stageName='Desenvolvimento',
    description='API/Lambda - Versao Desenvolvimento'
)
print("Estágio 'Desenvolvimento' implantado.")

# 12. Atualizar integração para alias 'prod'
lambda_arn_prod = f'arn:aws:lambda:{region}:SEU_ID_DE_CONTA:function:{function_name}:prod'
apigateway_client.update_integration(
    restApiId=api_id,
    resourceId=hello_id,
    httpMethod='GET',
    patchOperations=[
        {
            'op': 'replace',
            'path': '/uri',
            'value': f'arn:aws:apigateway:{region}:lambda:path/2015-03-31/functions/{lambda_arn_prod}/invocations'
        }
    ]
)
print("Integração atualizada para alias 'prod'.")

# 13. Implantar estágio Producao
apigateway_client.create_deployment(
    restApiId=api_id,
    stageName='Producao',
    description='API/Lambda - Producao'
)
print("Estágio 'Producao' implantado.")
